In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./index.md
./module-p8-2.ipynb
./forecast.txt
./module-p8.ipynb
./data/stores.csv
./data/oil.csv
./data/transactions.csv
./data/holidays_events.csv
./data/train.csv
./data/train_filled.csv
./data/test.csv
./data/sample_submission.csv


In [3]:
#!pip install "gluonts[mxnet,pro]"

## EDA

In [4]:
from datetime import datetime

df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')
df_test.date.min()
df_test.date.max()
min = datetime.strptime(df_test.date.min(), '%Y-%M-%d')
max = datetime.strptime(df_test.date.max(), '%Y-%M-%d')
prediction_length = (max - min).days


'2017-08-16'

'2017-08-31'

In [5]:
df_train
df_train.shape
df_train.date.min()
df_train.date.max()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


(3000888, 6)

'2013-01-01'

'2017-08-15'

dataset starts from 2013-01-01 to 2017-08-15

predictions must be made from 2017-08-16 to 2017-08-31


In [6]:
df_train.dtypes

id               int64
date            object
store_nbr        int64
family          object
sales          float64
onpromotion      int64
dtype: object

Do we have continuous dates ?

no : 25 december is missing for all


In [7]:
df_train.groupby(by=['date']).count()

df_train[df_train['date'] == '2013-12-24']
df_train[df_train['date'] == '2013-12-25']

,id,store_nbr,family,sales,onpromotion
date,,,,,
2013-01-01,1782,1782,1782,1782,1782
2013-01-02,1782,1782,1782,1782,1782
2013-01-03,1782,1782,1782,1782,1782
2013-01-04,1782,1782,1782,1782,1782
2013-01-05,1782,1782,1782,1782,1782
...,...,...,...,...,...
2017-08-11,1782,1782,1782,1782,1782
2017-08-12,1782,1782,1782,1782,1782
2017-08-13,1782,1782,1782,1782,1782


,id,date,store_nbr,family,sales,onpromotion
636174,636174,2013-12-24,1,AUTOMOTIVE,1.000,0
636175,636175,2013-12-24,1,BABY CARE,0.000,0
636176,636176,2013-12-24,1,BEAUTY,2.000,0
636177,636177,2013-12-24,1,BEVERAGES,1248.000,0
636178,636178,2013-12-24,1,BOOKS,0.000,0
...,...,...,...,...,...,...
637951,637951,2013-12-24,9,POULTRY,848.844,0
637952,637952,2013-12-24,9,PREPARED FOODS,40.000,0
637953,637953,2013-12-24,9,PRODUCE,0.000,0
637954,637954,2013-12-24,9,SCHOOL AND OFFICE SUPPLIES,0.000,0


,id,date,store_nbr,family,sales,onpromotion


In [8]:
# add missing dates to train dataset : required for fitting the model

un_store = df_train['store_nbr'].unique()
un_family = df_train['family'].unique()

tmp_df = pd.DataFrame(
    columns=['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion'],
    )

for i, s in enumerate(un_store):
    for j, f in enumerate(un_family):
        tmp_df = tmp_df.append(
            pd.DataFrame(
                {'id': 0, 'date': '2013-12-25', 'store_nbr': s, 'family': f, 'sales': 0, 'onpromotion' : 0},
                index=[0]),
            ignore_index=True,
        )
        tmp_df = tmp_df.append(
            pd.DataFrame(
                {'id': 0, 'date': '2014-12-25', 'store_nbr': s, 'family': f, 'sales': 0, 'onpromotion' : 0},
                index=[0]),
            ignore_index=True
        )
        tmp_df = tmp_df.append(
            pd.DataFrame(
                {'id': 0, 'date': '2015-12-25', 'store_nbr': s, 'family': f, 'sales': 0, 'onpromotion' : 0},
                index=[0]),
            ignore_index=True
        )
        tmp_df = tmp_df.append(
            pd.DataFrame(
                {'id': 0, 'date': '2016-12-25', 'store_nbr': s, 'family': f, 'sales': 0, 'onpromotion' : 0},
                index=[0]),
            ignore_index=True
        )

print(tmp_df)
df_train = df_train.append(tmp_df)
df_train.to_csv('data/train_filled.csv')


     id        date store_nbr                      family sales onpromotion
0     0  2013-12-25         1                  AUTOMOTIVE     0           0
1     0  2014-12-25         1                  AUTOMOTIVE     0           0
2     0  2015-12-25         1                  AUTOMOTIVE     0           0
3     0  2016-12-25         1                  AUTOMOTIVE     0           0
4     0  2013-12-25         1                   BABY CARE     0           0
...  ..         ...       ...                         ...   ...         ...
7123  0  2016-12-25         9  SCHOOL AND OFFICE SUPPLIES     0           0
7124  0  2013-12-25         9                     SEAFOOD     0           0
7125  0  2014-12-25         9                     SEAFOOD     0           0
7126  0  2015-12-25         9                     SEAFOOD     0           0
7127  0  2016-12-25         9                     SEAFOOD     0           0

[7128 rows x 6 columns]


In [9]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df_train['family_cat'] = le.fit_transform(df_train['family'])
le.classes_

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR WINE BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

In [10]:
nb_stores = df_train['store_nbr'].unique()
nb_stores

array([1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29, 3, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 4,
       40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 5, 50, 51, 52, 53, 54, 6,
       7, 8, 9], dtype=object)

In [11]:
df_test['family_cat'] = le.fit_transform(df_test['family'])
df_test

,id,date,store_nbr,family,onpromotion,family_cat
0,3000888,2017-08-16,1,AUTOMOTIVE,0,0
1,3000889,2017-08-16,1,BABY CARE,0,1
2,3000890,2017-08-16,1,BEAUTY,2,2
3,3000891,2017-08-16,1,BEVERAGES,20,3
4,3000892,2017-08-16,1,BOOKS,0,4
...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,28
28508,3029396,2017-08-31,9,PREPARED FOODS,0,29
28509,3029397,2017-08-31,9,PRODUCE,1,30
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,31


In [12]:
df_train.set_index('date', inplace=True)
df_train.sort_index(inplace=True)

In [13]:
df_test.set_index('date', inplace=True)
df_test

,id,store_nbr,family,onpromotion,family_cat
date,,,,,
2017-08-16,3000888,1,AUTOMOTIVE,0,0
2017-08-16,3000889,1,BABY CARE,0,1
2017-08-16,3000890,1,BEAUTY,2,2
2017-08-16,3000891,1,BEVERAGES,20,3
2017-08-16,3000892,1,BOOKS,0,4
...,...,...,...,...,...
2017-08-31,3029395,9,POULTRY,1,28
2017-08-31,3029396,9,PREPARED FOODS,0,29
2017-08-31,3029397,9,PRODUCE,1,30


In [14]:
df_test['sales'] = 0

In [15]:
df_train.loc['2013-12-25']

,id,store_nbr,family,sales,onpromotion,family_cat
date,,,,,,
2013-12-25,0,41,PREPARED FOODS,0,0,29
2013-12-25,0,42,BOOKS,0,0,4
2013-12-25,0,42,BEVERAGES,0,0,3
2013-12-25,0,42,BEAUTY,0,0,2
2013-12-25,0,42,BABY CARE,0,0,1
...,...,...,...,...,...,...
2013-12-25,0,25,PET SUPPLIES,0,0,26
2013-12-25,0,25,PERSONAL CARE,0,0,25
2013-12-25,0,26,BEVERAGES,0,0,3


In [16]:
# df_train

In [17]:
# df_test

In [18]:
df_test.groupby(by=['store_nbr', 'family']).count()

id  onpromotion  family_cat  sales
store_nbr family                                                        
1         AUTOMOTIVE                  16           16          16     16
          BABY CARE                   16           16          16     16
          BEAUTY                      16           16          16     16
          BEVERAGES                   16           16          16     16
          BOOKS                       16           16          16     16
...                                   ..          ...         ...    ...
54        POULTRY                     16           16          16     16
          PREPARED FOODS              16           16          16     16
          PRODUCE                     16           16          16     16
          SCHOOL AND OFFICE SUPPLIES  16           16          16     16
          SEAFOOD                     16           16          16     16

[1782 rows x 4 columns]

The prediction length is for 16 days

## GluonTS

In [19]:
from gluonts.dataset.common import ListDataset
from gluonts.model.lstnet import LSTNetEstimator
from gluonts.model.deepar import DeepAREstimator
from gluonts.dataset.pandas import PandasDataset


from gluonts.mx import Trainer

freq = "D"
# start = pd.Period("2013-01-01", freq=freq)
# prediction_length = 200 # predict this many points
context_length = 2000 # train on this number of points
prediction_length = 16 # see prediction lenght infered above

#### Dataset Preparation

In [20]:
# df_train['dataset'] = 'train'
# df_test['dataset'] = 'test'
# df_all = pd.concat([df_train, df_test])
# df_all

In [21]:

df = df_train [['store_nbr', 'family', 'sales', 'id']]
df = df [df['store_nbr'] == 1]
df = df.astype({'store_nbr': 'string', 'family':'string', 'sales': 'int'})
df.dtypes
df

store_nbr    string
family       string
sales         int64
id           object
dtype: object

,store_nbr,family,sales,id
date,,,,
2013-01-01,1,AUTOMOTIVE,0,0
2013-01-01,1,MAGAZINES,0,23
2013-01-01,1,LIQUOR WINE BEER,0,22
2013-01-01,1,LINGERIE,0,21
2013-01-01,1,LAWN AND GARDEN,0,20
...,...,...,...,...
2017-08-15,1,SEAFOOD,22,2999138
2017-08-15,1,SCHOOL AND OFFICE SUPPLIES,0,2999137
2017-08-15,1,PRODUCE,2240,2999136


In [22]:
np.unique(df['family']).size

33

In [23]:

ds = PandasDataset.from_long_dataframe(df, freq='d', target='sales', item_id=['family'])
# ds_train = PandasDataset.from_long_dataframe(df[:-prediction_length], freq='d', target='sales', item_id=['family'])
# ds_test = PandasDataset.from_long_dataframe(df.tail(prediction_length), freq='d', target='sales', item_id=['family'])


In [24]:
# ds_train

In [25]:
# ds_test 

In [21]:
# do the same dataset preparation for all the 'nb_stores' stores

all_stores_ds = []

for store in nb_stores:
    df_tmp = df_train [['store_nbr', 'family', 'sales', 'id']]
    df_tmp = df_tmp [df_tmp['store_nbr'] == store]
    df_tmp = df_tmp.astype({'store_nbr': 'string', 'family':'string', 'sales': 'int'})
    ds_tmp = PandasDataset.from_long_dataframe(df_tmp, freq='d', target='sales', item_id='family')
    all_stores_ds.append({'store_nbr': store, 'ds': ds_tmp})

# all_stores_ds

In [22]:
all_stores_ds[0]['store_nbr']
all_stores_ds[0]['ds']


1

PandasDataset(dataframes={'AUTOMOTIVE':            store_nbr      family  sales       id
date                                            
2013-01-01         1  AUTOMOTIVE      0        0
2013-01-02         1  AUTOMOTIVE      2     1782
2013-01-03         1  AUTOMOTIVE      3     3564
2013-01-04         1  AUTOMOTIVE      3     5346
2013-01-05         1  AUTOMOTIVE      5     7128
...              ...         ...    ...      ...
2017-08-11         1  AUTOMOTIVE      1  2991978
2017-08-12         1  AUTOMOTIVE      6  2993760
2017-08-13         1  AUTOMOTIVE      1  2995542
2017-08-14         1  AUTOMOTIVE      1  2997324
2017-08-15         1  AUTOMOTIVE      4  2999106

[1688 rows x 4 columns], 'BABY CARE':            store_nbr     family  sales       id
date                                           
2013-01-01         1  BABY CARE      0        1
2013-01-02         1  BABY CARE      0     1783
2013-01-03         1  BABY CARE      0     3565
2013-01-04         1  BABY CARE      0     5

#### Model Training

In [23]:
from gluonts.evaluation import make_evaluation_predictions

def show_predictor(predictor):
    prediction = next(predictor.predict(ds))
    print(prediction.mean)
    prediction.plot()

def get_forecasts(ds):
    estimator = DeepAREstimator(
        freq=ds.freq, 
        prediction_length=prediction_length, 
        trainer=Trainer(
            epochs=5,
            learning_rate=1e-3,
            num_batches_per_epoch=10,
            patience=2
            )
    )
    predictor = estimator.train(ds)
    # show_predictor(predictor)
    forecast_it, ts_it = make_evaluation_predictions(
        dataset=ds,  # test dataset
        predictor=predictor,  # predictor
        num_samples=10,  # number of sample paths we want for evaluation
    )
    tss = list(ts_it)
    forecasts = list(forecast_it)
    return tss, forecasts



In [ ]:
# sample usage
# tss, forecasts = get_forecasts(ds)

In [24]:
all_stores_forecasts = []

for ds in all_stores_ds:
    store_nbr = ds['store_nbr']
    ds = ds['ds']
    tss, forecasts = get_forecasts(ds)
    all_stores_forecasts.append({'store_nbr':store_nbr, 'forecasts': forecasts})

100%|██████████| 10/10 [00:00<00:00, 20.72it/s, epoch=5/5, avg_epoch_loss=4.9]


In [27]:
all_stores_forecasts[0]['store_nbr']
all_stores_forecasts[0]['forecasts'][0].item_id

all_stores_forecasts[27]['store_nbr']
all_stores_forecasts[27]['forecasts'][0].mean

1

'AUTOMOTIVE'

34

array([2.67705  , 4.9726324, 4.1175594, 2.1599407, 5.0596776, 4.4404154,
       2.6668267, 2.0827475, 3.5493999, 4.2353215, 1.8752264, 2.9581506,
       2.4157844, 2.524041 , 2.422216 , 2.7825336], dtype=float32)

In [75]:
def show_forecasts(tss, forecasts):
    # first entry of the time series list
    len(tss[0])
    ts_entry = tss[0]
    ts_entry.tail(20)

    forecast_entry = forecasts[0]
    print(f"Number of sample paths: {forecast_entry.num_samples}")  
    print(f"Dimension of samples: {forecast_entry.samples.shape}")
    print(f"Start date of the forecast window: {forecast_entry.start_date}")
    print(f"Frequency of the time series: {forecast_entry.freq}")

    print(f"Mean of the future window:\n {forecast_entry.mean}")
    print(f"0.5-quantile (median) of the future window:\n {forecast_entry.quantile(0.5)}")
    

#### Predictions

In [ ]:
# len(df_test['store_nbr'].unique())
# len(df_test['family'].unique())

In [ ]:
# len(forecasts)
# forecasts[0].item_id
# forecasts[0].mean
# categories_arr = [cast.item_id for cast in forecasts ]

In [ ]:
# Update test dataset with predicted values

# categories_arr = [cast.item_id for cast in forecasts ]

# for index, cat in enumerate(categories_arr):
#     df_test.loc[(df_test['store_nbr']==1) & (df_test['family']==cat), 'sales'] = forecasts[index].mean

# df_test

In [ ]:
# for index, cast in enumerate(all_stores_forecasts):
#     all_stores_forecasts[index]['store_nbr']

In [33]:
def fill_df_with_predictions(df, all_forecasts):
    df_tmp = df.copy()
    
    for index, cast in enumerate(all_forecasts):
        store_nbr = all_forecasts[index]['store_nbr']
        store_fcasts = all_forecasts[index]['forecasts']
        categories_arr = [cast.item_id for cast in store_fcasts ]
        
        for index, cat in enumerate(categories_arr):
            cat_fcasts = store_fcasts[index]
            cat = cat_fcasts.item_id
            df_tmp.loc[(df_tmp['store_nbr']==store_nbr) & (df_tmp['family']==cat), 'sales'] = cat_fcasts.mean
    
    return df_tmp
        


In [35]:

df_tmp = fill_df_with_predictions(df_test, all_stores_forecasts)


In [38]:

df_tmp[(df_tmp['store_nbr'] == 1) & (df_tmp['family'] == 'BABY CARE')]

,id,store_nbr,family,onpromotion,family_cat,sales
date,,,,,,
2017-08-16,3000889,1,BABY CARE,0,1,6.268941
2017-08-17,3002671,1,BABY CARE,0,1,14.494840
2017-08-18,3004453,1,BABY CARE,0,1,8.861006
2017-08-19,3006235,1,BABY CARE,0,1,-3.199225
2017-08-20,3008017,1,BABY CARE,0,1,5.781116
2017-08-21,3009799,1,BABY CARE,0,1,-6.387799
2017-08-22,3011581,1,BABY CARE,0,1,-20.461315
2017-08-23,3013363,1,BABY CARE,0,1,8.127833
2017-08-24,3015145,1,BABY CARE,0,1,12.777693


In [41]:
df_tmp[['id', 'sales']].to_csv('submission.csv', index=False)

#### Evaluation of Performance

In [43]:
from gluonts.evaluation import Evaluator

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(tss, forecasts)
agg_metrics['RMSE']
item_metrics

Running evaluation: 33it [00:01, 32.57it/s]
/home/vincent/devel/vjuge/oc-iml/venv/lib64/python3.9/site-packages/pandas/core/dtypes/cast.py:1201: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)


412.3754035360752

,item_id,MSE,abs_error,abs_target_sum,abs_target_mean,seasonal_error,MASE,MAPE,sMAPE,ND,MSIS,QuantileLoss[0.1],Coverage[0.1],QuantileLoss[0.5],Coverage[0.5],QuantileLoss[0.9],Coverage[0.9]
0,AUTOMOTIVE,3.631356e+01,85.888611,229.0,14.3125,8.113106,0.661650,0.373358,0.420924,0.375059,4.554395,33.577699,0.0625,85.888610,0.2500,39.604323,0.7500
1,BABY CARE,1.058914e+00,8.601103,8.0,0.5000,0.292041,1.840733,0.945171,1.948598,1.075138,61.804068,2.289154,0.4375,8.601103,0.7500,13.671977,0.7500
2,BEAUTY,2.901235e+01,74.277359,153.0,9.5625,3.209455,1.446456,0.477902,0.577546,0.485473,18.029520,25.344522,0.0625,74.277362,0.1875,69.037933,0.3750
3,BEVERAGES,1.265714e+06,13274.930664,59382.0,3711.3750,814.184919,1.019035,0.199417,0.233063,0.223551,7.929214,5475.440674,0.0000,13274.930420,0.1250,6882.679004,0.6875
4,BOOKS,2.357270e+03,730.862915,0.0,0.0000,0.000000,inf,NaN,2.000000,inf,inf,448.406790,0.0000,730.862908,0.0625,185.799277,0.9375
5,BREAD/BAKERY,1.140215e+04,1162.589844,7880.0,492.5000,134.592460,0.539866,0.156489,0.151727,0.147537,5.732672,716.644662,0.0625,1162.589844,0.5000,789.608374,0.9375
6,CELEBRATION,3.673980e+01,87.205322,238.0,14.8750,6.116697,0.891058,0.446622,0.393909,0.366409,5.764933,45.271253,0.1250,87.205329,0.3750,34.749428,0.9375
7,CLEANING,7.294219e+05,8820.925781,30748.0,1921.7500,415.828247,1.325807,0.225234,0.272537,0.286878,17.251147,2837.271375,0.0000,8820.925293,0.1875,7916.937671,0.6875
8,DAIRY,6.101988e+04,2330.289551,13238.0,827.3750,214.272292,0.679710,0.173936,0.184546,0.176030,5.350798,1211.736578,0.0625,2330.289734,0.1875,1352.100891,0.9375
9,DELI,1.894466e+04,1627.813110,9179.0,573.6875,152.503291,0.667122,0.163324,0.180200,0.177341,5.945618,694.231879,0.0625,1627.813141,0.2500,982.020593,0.6875


GluonTS multivariate :

<https://github.com/awslabs/gluonts/issues/494>

<https://github.com/awslabs/gluonts/issues/517>

<https://ts.gluon.ai/stable/tutorials/forecasting/extended_tutorial.html>



In [ ]:
from gluonts.evaluation import Evaluator
from gluonts.evaluation import make_evaluation_predictions
import matplotlib.pyplot as plt
import json
import numpy as np

def make_predictions(_predictor, ds, num_samples=100):
    forecast_it, ts_it = make_evaluation_predictions(
        dataset=ds,  # test dataset
        predictor=_predictor,  # predictor
        num_samples=num_samples,  # number of sample paths we want for evaluation
    )
    return list(forecast_it), list(ts_it)

def plot_prob_forecasts(ts_entry, forecast_entry):
    plot_length = context_length + prediction_length
    prediction_intervals = (50.0, 90.0)
    legend = ["observations", "median prediction"] + [ f"{k}% prediction interval" for k in prediction_intervals][::-1]

    fig, ax = plt.subplots(1, 1, figsize=(40, 20))
    ts_entry[-plot_length:].plot(ax=ax)  # plot the time series
    forecast_entry.plot(prediction_intervals=prediction_intervals, color="cadetblue")
    plt.axvline(forecast_entry.start_date, color="g", lw=1) #end of train dataset
    plt.grid(which="major")
    plt.legend(legend, loc="upper left")
    plt.show()

def gluonts_metrics(tss, forecasts):
    evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
    agg_metrics, item_metrics = evaluator(tss, forecasts)

    print(json.dumps(agg_metrics, indent=4))

def train_and_predict(dataset, estimator):
    predictor = estimator.train(dataset)
    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset, predictor=predictor
    )
    evaluator = Evaluator(quantiles=(np.arange(20) / 20.0)[1:])
    agg_metrics, item_metrics = evaluator(ts_it, forecast_it, num_series=len(dataset))
    return agg_metrics["RMSE"]

## Prophet

In [ ]:
from prophet import Prophet

m = Prophet()
df_automotive['ds'] = df_automotive.index
df_automotive['y'] = df_automotive['sales']

df_automotive

In [ ]:
m.fit(df_automotive[:100000])

In [ ]:
future = m.make_future_dataframe(periods=365)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()


In [ ]:
fig1 = m.plot(forecast, figsize=(15,10))


In [ ]:
fig2 = m.plot_components(forecast, figsize=(15,10))


In [ ]:
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)
